<a href="https://colab.research.google.com/github/karthikRavichandran/News_classfication/blob/master/News_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, Embedding, Conv1D, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow import math
import tqdm
import re
import string

In [0]:

# d_f = pd.read_csv("/content/news.csv", sep="\t")


In [2]:
# reading data from News sentiment analysis data set downloaded from kaggle 
d_f2 = pd.read_csv("/content/drive/My Drive/MODELS/all-data.csv")
d_f2.columns = ["classes", "News"]
d_f2["classes_num"]= d_f2.classes.map({'neutral': 1, 'negative': 0, 'positive' : 2})
d_f2["Positive"] = np.where(d_f2["classes"].str.contains("positive"), 1,0)
d_f2["negative"] = np.where(d_f2["classes"].str.contains("negative"), 1,0)
d_f2["neutral"] = np.where(d_f2["classes"].str.contains("neutral"), 1,0)
d_f2.head()

,classes,News,classes_num,Positive,negative,neutral
0,neutral,Technopolis plans to develop in stages an area...,1,0,0,1
1,negative,The international electronic industry company ...,0,0,1,0
2,positive,With the new production plant the company woul...,2,1,0,0
3,positive,According to the company 's updated strategy f...,2,1,0,0
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,2,1,0,0


In [3]:
np.unique(d_f2["classes"], return_counts = True) #class bias analysis

(array(['negative', 'neutral', 'positive'], dtype=object),
 array([ 604, 2878, 1363]))

In [4]:
#text preprocessing 
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation.
      Input to the clean_text_round1 a sentance with spl char
      Output is a clean text by removing spl char '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', 'numberstring', text)
    return text
for i in tqdm.tqdm(range(d_f2['News'].shape[0])):
#     print(i)
    d_f2['News'].iloc[i] = clean_text_round1(d_f2['News'].iloc[i])

  0%|          | 0/4845 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 4845/4845 [00:01<00:00, 4213.14it/s]


In [8]:
learn_text = " [%] (I am?) in .... ////r><><> march2020?"
learn_text = clean_text_round1(learn_text)

print(learn_text) 

  i am in  r numberstring


In [9]:
#Reading data from IMDB comment sentiment analysis data set
df = pd.read_csv("/content/drive/My Drive/MODELS/train.csv",sep="\t" )
df["Positive"] = (df["Sentiment"] > 2).astype(int)
df["Negative"] = (df["Sentiment"] < 2).astype(int)
df["neutral"] = (df["Sentiment"] == 2).astype(int)
train2 = df["Phrase"]
train2_Y = df.drop(["PhraseId", "SentenceId", "Phrase", "Sentiment"], axis =1).values
for i in tqdm.tqdm(range(train2.shape[0])):
#     print(i)
    train2.iloc[i] = clean_text_round1(train2.iloc[i])

  0%|          | 0/156060 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 156060/156060 [02:10<00:00, 1192.58it/s]


In [10]:
d_f2.head()

,classes,News,classes_num,Positive,negative,neutral
0,neutral,technopolis plans to develop in stages an area...,1,0,0,1
1,negative,the international electronic industry company ...,0,0,1,0
2,positive,with the new production plant the company woul...,2,1,0,0
3,positive,according to the company s updated strategy fo...,2,1,0,0
4,positive,financing of aspocomp s growth aspocomp is agg...,2,1,0,0


In [11]:
# Adding both the data sets together in a dataframe
three_class = d_f2.drop(["News", "classes_num", "classes",], axis = 1).values
print(three_class.shape)
three_class = np.append(three_class, train2_Y, axis =0)
print(np.shape(three_class))
News_max = np.append(d_f2['News'], train2, axis = 0)
print(np.shape(News_max))

(4845, 3)
(160905, 3)
(160905,)


In [34]:
#Splitting data set for train and test
df_train_max, df_test, Y_multi_train_max, Y_multi_test = train_test_split(News_max, three_class, test_size=0.33, random_state = 42)
print("Train shape : ", df_train_max.shape, "test shape : ", df_test.shape)
print("Train shape : ", Y_multi_train_max.shape, "test shape : ", Y_multi_test.shape)
print("Class balance : ",Y_multi_train_max[:,0].sum(),Y_multi_train_max[:,1].sum(), Y_multi_train_max[:,2].sum() )
# print(Y_multi_train_max[:,0])

Train shape :  (107806,) test shape :  (53099,)
Train shape :  (107806, 3) test shape :  (53099, 3)
Class balance :  29161 23373 55272
[0 0 1 ... 0 0 0]


In [36]:
#further splitting train_max into validation and train. 
df_train, df_val, Y_multi_train, Y_multi_val = train_test_split(df_train_max, Y_multi_train_max, test_size=0.33, random_state = 42)
print("Train shape : ", df_train.shape, "val shape : ", df_val.shape)
print("Train shape : ", Y_multi_train.shape, "val shape : ", Y_multi_val.shape)

Train shape :  (72230,) val shape :  (35576,)
Train shape :  (72230, 3) val shape :  (35576, 3)


In [39]:
# text preprocessing for converting text in to vector
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)# fit on training data
sequences_train = tokenizer.texts_to_sequences(df_train)# transform on fitted Tokenizer
sequences_test = tokenizer.texts_to_sequences(df_test)# transform on fitted Tokenizer
sequences_val = tokenizer.texts_to_sequences(df_val)# transform on fitted Tokenizer
max_ = 0
max_ = max([len(seq) for seq in sequences_train])
word2idx = tokenizer.word_index
V = len(word2idx)
print("length of max sequence : ", max_)
print('Found %s unique tokens.' % V)
file_path = "/content/drive/My Drive/MODELS/"
print('saving tokenizer to json file =======> ', file_path)
tokenizer_json = tokenizer.to_json()
with open(file_path +'/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

print("Loading file tokenizer from json =======> ", file_path +'/tokenizer.json')
with open(file_path + '/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

length of max sequence :  52
Found 19519 unique tokens.
saving tokenizer to json file =======>  /content/drive/My Drive/MODELS/
Loading file tokenizer from json =======>  /content/drive/My Drive/MODELS//tokenizer.json


In [40]:
# pad sequences zeros
data_train = pad_sequences(sequences_train, maxlen = max_)
print('Shape of data train tensor:', data_train.shape)
data_test = pad_sequences(sequences_test, maxlen = max_)
print('Shape of data train tensor:', data_test.shape)
data_val = pad_sequences(sequences_val, maxlen = max_)
print('Shape of data train tensor:', data_val.shape)

Shape of data train tensor: (72230, 52)
Shape of data train tensor: (53099, 52)
Shape of data train tensor: (35576, 52)


In [0]:
import tensorflow.keras.backend as K
from sklearn.utils import class_weight
# class_weight = class_weight.compute_class_weight('balanced'
#                                                ,np.unique(Ytrain)
#                                                ,Ytrain)

def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val 
# Here I used f1 score in metrics


In [44]:
#Download glove pretrained weights
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-28 16:28:26--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-28 16:28:26--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-28 16:28:27--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [45]:
#unzip the file from downloaded glove zip file
from zipfile import ZipFile   
# specifying the zip file name 
file_name = "/content/glove.6B.zip"
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
glove.6B.50d.txt                               2014-08-04 13:15:00    171350079
glove.6B.100d.txt                              2014-08-04 13:14:34    347116733
glove.6B.200d.txt                              2014-08-04 13:14:44    693432828
glove.6B.300d.txt                              2014-08-27 12:19:16   1037962819
Extracting all the files now...
Done!


In [46]:
# load glove pretained weights in to dict with word and its vector rep.
import os
embeddings_index = {}
GLOVE_DIR = "/content/"
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

word_index = word2idx
print(len(word2idx))
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.
19519


In [0]:
# use the glove weight to the embedding layer (transfer learning)
from tensorflow.keras.layers import Embedding
MAX_SEQUENCE_LENGTH = T
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True) #trainable can be freezed if we want

In [0]:
# # Checking GPU !
# !cat /proc/meminfo
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [51]:
from tensorflow.keras.callbacks import ModelCheckpoint

T = data_train.shape[1] # or max_
# embedding dimensionality
D = 100 
# Hidden state dimensionality
H = 200
#==================================== tried different arch =============================

# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)  # global max pooling
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# preds = Dense(len(labels_index), activation='softmax')(x)
#==========================================================================================


i = Input(shape=(T,))
x = embedding_layer(i) # we can use glove pretrained weights for this embedding(transfer learning)
                            # freez the weights
x = LSTM(H, return_sequences=True)(x)
x = Bidirectional(LSTM(100, return_sequences=True, dropout = 0.50), merge_mode='concat')(x)
# x = LSTM(50, return_sequences=True)(x)
# x = Conv1D(32, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(64, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(3, activation='softmax')(x)
# x = Dropout(0.2)(x)

model = Model(i, x)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1]) # [get_f1] 

# filepath = "/content/"
filepath="/content/weights-improvement-f1-{epoch:02d}-{val_get_f1:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_get_f1', 
                             verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
r = model.fit(
  # np.append(data_train, data_test, axis = 0),
  # np.append(Y_multi_train, Y_multi_test, axis =0),
  data_train, Y_multi_train,
  epochs=10,
  validation_data=(data_val, np.array(Y_multi_val),),
  batch_size=64, verbose=1, shuffle=True, callbacks=callbacks_list
)

Epoch 1/10
1129/1129 [==============================] - ETA: 0s - loss: 0.6962 - get_f1: 0.6617
Epoch 00001: val_get_f1 improved from -inf to 0.72521, saving model to /content/weights-improvement-f1-01-0.73.hdf5
1129/1129 [==============================] - 38s 33ms/step - loss: 0.6962 - get_f1: 0.6617 - val_loss: 0.6321 - val_get_f1: 0.7252
Epoch 2/10
1129/1129 [==============================] - ETA: 0s - loss: 0.5764 - get_f1: 0.7556
Epoch 00002: val_get_f1 improved from 0.72521 to 0.72870, saving model to /content/weights-improvement-f1-02-0.73.hdf5
1129/1129 [==============================] - 36s 32ms/step - loss: 0.5764 - get_f1: 0.7556 - val_loss: 0.6288 - val_get_f1: 0.7287
Epoch 3/10
1129/1129 [==============================] - ETA: 0s - loss: 0.5171 - get_f1: 0.7817
Epoch 00003: val_get_f1 did not improve from 0.72870
1129/1129 [==============================] - 36s 32ms/step - loss: 0.5171 - get_f1: 0.7817 - val_loss: 0.6494 - val_get_f1: 0.7274
Epoch 4/10
1129/1129 [=========

In [0]:
predicted_test = model.predict(data_test)>0.5 #predicting test data in trained model
predicted_test = predicted_test*1
predicted_val = model.predict(data_val)>0.5 #predicting test data in trained model
predicted_val = predicted_val*1


In [78]:
#Inspecting confusion matrix for val data set as well as test data set
val_con = math.confusion_matrix(Y_multi_test.argmax(axis = 1), predicted_test.argmax(axis = 1)) 
test_con = math.confusion_matrix(Y_multi_val.argmax(axis = 1), predicted_val.argmax(axis = 1)) 
print(val_con)
print(test_con)

tf.Tensor(
[[10305   368  3662]
 [  599  8605  2372]
 [ 3651  3776 19761]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[ 6832   237  2493]
 [  379  5643  1738]
 [ 2548  2580 13126]], shape=(3, 3), dtype=int32)


In [0]:
def prediction_(text, model, T):
    sequences_test_2 = tokenizer.texts_to_sequences([text])
    sequences_test_2 = pad_sequences(sequences_test_2, maxlen = max_)
    print("prediction : ", model.predict(sequences_test_2))
    return np.argmax(model.predict(sequences_test_2))

In [86]:
test = "all the days the stock went low and it did not improve even at the end of the day"
test = clean_text_round1(test)
# print(test)
print(test)
test_sen = np.argmax(prediction_(test, model, T))
print(test_sen)
if test_sen == 0:
  print("Postive sentance ")
elif test_sen == 1:
  print("negative sentance")
else:
  print("neutral")
# predict of the above sentance in negative and it's true.

all the days the stock went low and it did not improve even at the end of the day
prediction :  [[6.2096942e-06 8.6855495e-01 1.3143887e-01]]
0
Postive sentance 
